# Advanced Practice: Instance Methods & Encapsulation

This notebook contains **advanced problems with solutions** based on the concepts of:

- Instance methods
- Encapsulation of data and behavior in classes
- Separating responsibilities between methods
- Working with small helper classes (`DataPoint`, `Circle`, `Forex`, etc.)

Each problem is followed by a **solution section**. Try to solve the problem first, then compare your approach with the proposed solution.

---

## Contents

1. `Circle` Geometry Utilities (mutating vs non-mutating instance methods)
2. `Rectangle` Class with Rich Instance Methods
3. Extending the `Forex` Class with Query & Aggregation Methods
4. Designing a Robust `SafeForex` Loader (validation + instance methods)

---

## Problem 1 – Advanced `Circle` Geometry Utilities

We previously implemented a simple `Circle` class with an `area` method and some mutating methods like `translate` and `scale`.

In this problem, you will design a richer `Circle` API using **instance methods** and a mixture of **mutating** and **non-mutating** behavior.

### Requirements

Implement a `Circle` class with the following behavior:

- Initialization:
  - `__init__(self, center_x, center_y, radius)`
  - `center` stored as a tuple `(x, y)`
  - `radius` stored as a non-negative `float`
  - If `radius < 0`, raise a `ValueError` with a helpful error message.

- Instance methods:
  1. `area(self)` → returns the area.
  2. `circumference(self)` → returns the circumference.
  3. `contains_point(self, x, y)` → `True` if the point `(x, y)` lies **inside or on** the circle, `False` otherwise.
  4. `translate(self, dx, dy)` → **mutates** the instance by shifting the center by `(dx, dy)` and returns `None`.
  5. `translated(self, dx, dy)` → **does not mutate** the instance. Returns a **new** `Circle` that is the translated version.
  6. `scale(self, factor)` → **mutates** the instance by scaling the radius by `factor`. If `factor <= 0`, raise `ValueError`.
  7. `scaled(self, factor)` → like `scale`, but non-mutating: returns a **new** scaled circle, leaving the original unchanged.

### Best Practices Hints

- Avoid duplicating code between `translate` / `translated` and `scale` / `scaled`.
- Keep validation logic in small private helper methods when useful (e.g. `_validate_radius`).
- Prefer clear, self-documenting names and add docstrings for public methods.

Implement the class in the cell below, together with a few basic tests (using `assert`) that exercise:

- radius validation
- mutating vs non-mutating methods
- `contains_point` logic

> Try to complete this before looking at the solution.

In [1]:
import math

class Circle:
    """Your implementation here.

    TODO:
    - store center as (x, y)
    - validate radius (non-negative)
    - implement area, circumference
    - implement contains_point, translate / translated, scale / scaled
    """
    def __init__(self, center_x, center_y, radius):
        pass

    # Add the rest of the methods here


# TODO: add a few basic tests using assert once you implement the class.
# Example structure (replace with real tests after you implement Circle):
# c = Circle(0, 0, 1.0)
# assert math.isclose(c.area(), math.pi)
# assert c.contains_point(0, 0) is True
# ...

### Solution 1 – Advanced `Circle` Geometry Utilities

Below is one possible solution that follows the requested behavior and best practices.

Key ideas:

- Use small private helpers (`_validate_radius`, `_validate_scale_factor`).
- Implement the non-mutating methods in terms of the constructor and current instance attributes.
- Implement the mutating methods once and reuse logic when possible.
- Add simple but meaningful tests using `assert`.

You can compare this solution with your own implementation and tests.

In [2]:
import math

class Circle:
    """Represents a circle in 2D space.

    Attributes
    ----------
    center : tuple[float, float]
        The (x, y) coordinates of the circle center.
    radius : float
        The non-negative radius of the circle.
    """

    def __init__(self, center_x: float, center_y: float, radius: float) -> None:
        self.center = (float(center_x), float(center_y))
        self._validate_radius(radius)
        self.radius = float(radius)

    # --------------------------
    # Validation helpers
    # --------------------------
    def _validate_radius(self, radius: float) -> None:
        if radius < 0:
            raise ValueError(f"radius must be non-negative, got {radius!r}")

    def _validate_scale_factor(self, factor: float) -> None:
        if factor <= 0:
            raise ValueError(f"scale factor must be positive, got {factor!r}")

    # --------------------------
    # Geometric queries
    # --------------------------
    def area(self) -> float:
        """Return the area of the circle."""
        return math.pi * (self.radius ** 2)

    def circumference(self) -> float:
        """Return the circumference of the circle."""
        return 2 * math.pi * self.radius

    def contains_point(self, x: float, y: float) -> bool:
        """Return True if the point (x, y) lies inside or on the circle.

        Uses the distance from the center to the point and compares it to
        the radius.
        """
        dx = x - self.center[0]
        dy = y - self.center[1]
        distance_sq = dx * dx + dy * dy
        return distance_sq <= self.radius ** 2

    # --------------------------
    # Mutating operations
    # --------------------------
    def translate(self, dx: float, dy: float) -> None:
        """Translate the circle by (dx, dy), mutating this instance."""
        x, y = self.center
        self.center = (x + dx, y + dy)

    def scale(self, factor: float) -> None:
        """Scale the circle radius by `factor`, mutating this instance.

        Raises
        ------
        ValueError
            If `factor <= 0`.
        """
        self._validate_scale_factor(factor)
        self.radius *= factor

    # --------------------------
    # Non-mutating operations
    # --------------------------
    def translated(self, dx: float, dy: float) -> "Circle":
        """Return a new translated circle, leaving this one unchanged."""
        x, y = self.center
        return Circle(x + dx, y + dy, self.radius)

    def scaled(self, factor: float) -> "Circle":
        """Return a new circle with scaled radius, leaving this one unchanged."""
        self._validate_scale_factor(factor)
        return Circle(self.center[0], self.center[1], self.radius * factor)

    # --------------------------
    # Debug / representation helper (optional but useful)
    # --------------------------
    def __repr__(self) -> str:  # not required, but handy when inspecting objects
        return f"Circle(center={self.center!r}, radius={self.radius!r})"


# --------------------------
# Basic tests (sanity checks)
# --------------------------

# radius validation
try:
    Circle(0, 0, -1)
except ValueError as ex:
    assert "radius" in str(ex)
else:
    raise AssertionError("Expected ValueError for negative radius")

c = Circle(0, 0, 1.0)
assert math.isclose(c.area(), math.pi)
assert math.isclose(c.circumference(), 2 * math.pi)

# contains_point
assert c.contains_point(0, 0) is True
assert c.contains_point(1, 0) is True  # on the boundary
assert c.contains_point(1.01, 0) is False

# translated vs translate
c2 = c.translated(2, 3)
assert c.center == (0.0, 0.0)
assert c2.center == (2.0, 3.0)

c.translate(-1, 1)
assert c.center == (-1.0, 1.0)

# scaled vs scale
c3 = c.scaled(2)
assert math.isclose(c.radius, 1.0)  # original unchanged by scaled
assert math.isclose(c3.radius, 2.0)

c.scale(0.5)
assert math.isclose(c.radius, 0.5)

try:
    c.scale(0)
except ValueError:
    pass
else:
    raise AssertionError("Expected ValueError for non-positive scale factor")

c, c2, c3  # show last values for visual inspection if run in a notebook

(Circle(center=(-1.0, 1.0), radius=0.5),
 Circle(center=(2.0, 3.0), radius=1.0),
 Circle(center=(-1.0, 1.0), radius=2.0))

## Problem 2 – `Rectangle` with Rich Instance Methods

Design a `Rectangle` class that represents axis-aligned rectangles in 2D.

A rectangle is defined by:
- A lower-left corner `(x, y)`
- A `width` and `height` (both strictly positive)

    
### Requirements

    
Implement the following behavior:

- Initialization:
  - `__init__(self, x, y, width, height)`
  - If `width <= 0` or `height <= 0`, raise a `ValueError`.

- Properties / simple methods:
  1. `area(self)` → rectangle area.
  2. `perimeter(self)` → perimeter.
  3. `right(self)` → x-coordinate of the right edge.
  4. `top(self)` → y-coordinate of the top edge.

- Geometric queries:
  5. `contains_point(self, x, y)` → `True` if point `(x, y)` is inside or on the border.
  6. `intersects(self, other)` → `True` if two rectangles overlap with positive area or touch at edges/corners.

- Mutating / non-mutating methods:
  7. `translate(self, dx, dy)` → mutates the rectangle position.
  8. `translated(self, dx, dy)` → returns a new translated rectangle.

### Hints

- For `intersects`, it may help to reason along each axis separately.
- Try to re-use logic via instance methods instead of re-computing coordinates by hand everywhere.
- Add docstrings.
- Add some tests that cover non-overlapping, edge-touching, and fully overlapping cases.

> Implement your solution in the next cell before checking the official solution.

In [3]:
class Rectangle:
    """Your implementation here.

    TODO:
    - validate width / height
    - implement area, perimeter, right, top
    - implement contains_point, intersects
    - implement translate / translated
    """
    def __init__(self, x, y, width, height):
        pass

    # Add required instance methods here


# TODO: write tests using `assert` once you implement Rectangle.
# Example (after implementation):
# r1 = Rectangle(0, 0, 2, 2)
# r2 = Rectangle(1, 1, 2, 2)
# assert r1.intersects(r2) is True


### Solution 2 – `Rectangle` with Rich Instance Methods

Below is one possible implementation.

Key points:

- Encapsulate the geometry and expose small helper methods like `right()` and `top()`.
- Implement `intersects` by checking the projections on the x and y axes.
- Respect mutating vs non-mutating semantics for `translate` / `translated`.
- Use simple `assert`-based tests to verify basic behavior.

You can experiment further by adding methods like `intersection(self, other)` that returns the overlapping rectangle, if any.

In [4]:
class Rectangle:
    """Axis-aligned rectangle in 2D.

    A rectangle is represented by its lower-left corner (x, y), width, and height.
    """

    def __init__(self, x: float, y: float, width: float, height: float) -> None:
        if width <= 0:
            raise ValueError(f"width must be positive, got {width!r}")
        if height <= 0:
            raise ValueError(f"height must be positive, got {height!r}")
        self.x = float(x)
        self.y = float(y)
        self.width = float(width)
        self.height = float(height)

    # --------------------------
    # Basic geometry
    # --------------------------
    def area(self) -> float:
        """Return the area of the rectangle."""
        return self.width * self.height

    def perimeter(self) -> float:
        """Return the perimeter of the rectangle."""
        return 2 * (self.width + self.height)

    def right(self) -> float:
        """Return the x-coordinate of the right edge."""
        return self.x + self.width

    def top(self) -> float:
        """Return the y-coordinate of the top edge."""
        return self.y + self.height

    # --------------------------
    # Queries
    # --------------------------
    def contains_point(self, x: float, y: float) -> bool:
        """Return True if the point (x, y) lies inside or on the border."""
        return (self.x <= x <= self.right()) and (self.y <= y <= self.top())

    def intersects(self, other: "Rectangle") -> bool:
        """Return True if this rectangle intersects or touches `other`.

        We consider touching at edges/corners as intersection.
        """
        # Separating axis theorem for axis-aligned rectangles.
        # If there is a gap along x OR y, they do not intersect.
        no_overlap_x = self.right() < other.x or other.right() < self.x
        no_overlap_y = self.top() < other.y or other.top() < self.y
        return not (no_overlap_x or no_overlap_y)

    # --------------------------
    # Transformations
    # --------------------------
    def translate(self, dx: float, dy: float) -> None:
        """Translate this rectangle by (dx, dy), mutating the instance."""
        self.x += dx
        self.y += dy

    def translated(self, dx: float, dy: float) -> "Rectangle":
        """Return a new translated rectangle, leaving this instance unchanged."""
        return Rectangle(self.x + dx, self.y + dy, self.width, self.height)

    def __repr__(self) -> str:
        return (
            f"Rectangle(x={self.x!r}, y={self.y!r}, width={self.width!r}, height={self.height!r})"
        )


# --------------------------
# Tests
# --------------------------

# Validation
for w, h in [(-1, 1), (1, 0), (0, 0)]:
    try:
        Rectangle(0, 0, w, h)
    except ValueError:
        pass
    else:
        raise AssertionError("Expected ValueError for non-positive width/height")

r1 = Rectangle(0, 0, 2, 3)
assert r1.area() == 6
assert r1.perimeter() == 2 * (2 + 3)
assert r1.right() == 2
assert r1.top() == 3

# contains_point
assert r1.contains_point(0, 0) is True  # corner
assert r1.contains_point(1, 1) is True  # inside
assert r1.contains_point(2, 3) is True  # opposite corner
assert r1.contains_point(2.1, 3) is False

# intersection tests
r2 = Rectangle(1, 1, 2, 2)
assert r1.intersects(r2) is True  # overlapping

r3 = Rectangle(2, 3, 1, 1)  # single corner touching
assert r1.intersects(r3) is True

r4 = Rectangle(3.1, 0, 1, 1)  # clearly to the right
assert r1.intersects(r4) is False

r5 = Rectangle(-1, -1, 0.5, 0.5)
assert r1.intersects(r5) is False

# translate vs translated
r6 = r1.translated(10, 10)
assert (r1.x, r1.y) == (0.0, 0.0)
assert (r6.x, r6.y) == (10.0, 10.0)

r1.translate(1, -2)
assert (r1.x, r1.y) == (1.0, -2.0)

r1, r2, r3, r4, r5, r6  # for visual inspection

(Rectangle(x=1.0, y=-2.0, width=2.0, height=3.0),
 Rectangle(x=1.0, y=1.0, width=2.0, height=2.0),
 Rectangle(x=2.0, y=3.0, width=1.0, height=1.0),
 Rectangle(x=3.1, y=0.0, width=1.0, height=1.0),
 Rectangle(x=-1.0, y=-1.0, width=0.5, height=0.5),
 Rectangle(x=10.0, y=10.0, width=2.0, height=3.0))

## Problem 3 – Extending `Forex` with Query & Aggregation Methods

Recall the `Forex` / `DataPoint` design:

- `Forex` loads data from a CSV file (e.g. `DEXUSEU.csv`)
- Each row is represented as a `DataPoint` with:
  - `date` (a `date` object)
  - `value` (a `Decimal`)

In this problem, you will extend this design by adding **instance methods** to the `Forex` class that make querying and aggregating the data easier.

### Starter Code

We'll use a slightly simplified version of the `DataPoint` and `Forex` classes from the lesson. We assume the CSV format is:

```text
DATE,VALUE
2015-01-01,1.2345
2015-01-02,1.2500
...
```

Missing values are represented by a dot (`.`) and should be skipped.

Run the cell below to set up the base classes, then extend `Forex` with the requested instance methods in the following cell.

> Note: These methods should **not** hard-code any specific file name; they should operate on `self.data` only.

In [5]:
import csv
from datetime import datetime, date
from decimal import Decimal
from pathlib import Path


class DataPoint:
    """Represents a single (date, value) pair for a time series."""

    def __init__(self, date_str: str, value_str: str) -> None:
        self.date = datetime.strptime(date_str, "%Y-%m-%d").date()
        self.value = Decimal(value_str)

    def __repr__(self) -> str:
        return f"DataPoint(date={self.date.isoformat()}, value={self.value})"


class Forex:
    """Loads and stores forex data from a CSV file.

    Attributes
    ----------
    file_name : str
        Path to the CSV file.
    data : list[DataPoint]
        Parsed list of data points.
    """

    def __init__(self, file_name: str) -> None:
        self.file_name = file_name
        self.data = self._process_data()

    def _process_data(self) -> list[DataPoint]:
        path = Path(self.file_name)
        if not path.exists():
            raise FileNotFoundError(f"File not found: {self.file_name!r}")

        points: list[DataPoint] = []
        with path.open() as f:
            reader = csv.reader(f)
            next(reader)  # skip header
            for date_str, value_str in reader:
                if value_str == ".":
                    continue
                points.append(DataPoint(date_str, value_str))
        return points


# If you have a DEXUSEU.csv file, you can instantiate Forex like this (optional):
# forex = Forex("DEXUSEU.csv")
# forex.data[:5]


### Requirements

Add the following instance methods to the `Forex` class:

1. `min_value(self)` → returns the **minimum** `Decimal` value in the series.
2. `max_value(self)` → returns the **maximum** `Decimal` value in the series.
3. `value_on(self, query_date)` → returns the `Decimal` value for the exact `query_date` (`date` instance), or `None` if not found.
4. `between(self, start_date, end_date)` → returns a **new list** of `DataPoint` objects whose dates are between `start_date` and `end_date` (inclusive).
5. `moving_average(self, window)` → returns a **list of `(date, avg)` tuples** where `avg` is the arithmetic mean of the last `window` values (as `Decimal`), aligned by the **last date** in each window. For example, for window = 3, the first result corresponds to `self.data[2]` and uses values of `self.data[0:3]`.

### Constraints

- Do **not** re-read the CSV file in these methods; they should operate on `self.data`.
- Use instance methods – avoid writing free functions.
- Prefer clear, small methods with docstrings.
- Handle corner cases (e.g. `window <= 0`, `window > len(self.data)`) by raising `ValueError`.

Implement the methods in the next cell, and (optionally) add tests that run only if the file exists (to avoid errors when the CSV is missing).

In [6]:
# Extend the Forex class with the requested instance methods.
# You can either:
# - re-define the Forex class here, or
# - use monkey-patching style (assign functions to Forex), but re-defining
#   is usually cleaner in a notebook.

# TODO: Re-define Forex here (copy/paste from above and add methods),
# or define new methods directly on the existing class.

from decimal import Decimal
from datetime import date

# Example structure (replace `pass` with your implementation):
# class Forex(Forex):
#     def min_value(self):
#         pass
#     ...

pass  # Remove this and add your implementation


### Solution 3 – Extended `Forex` with Query & Aggregation Methods

Below is one possible solution that re-defines the `Forex` class including the new instance methods.

Key ideas:

- Use Python built-ins like `min`/`max` with a `key` function when working with custom objects.
- Use list comprehensions to filter `DataPoint` ranges.
- For `moving_average`, iterate with indices and keep a running window.
- Use `Decimal` consistently to avoid mixing floats and decimals.

If you have `DEXUSEU.csv` available, the tests will run; otherwise they will be skipped gracefully.

In [7]:
import csv
from datetime import datetime, date
from decimal import Decimal
from pathlib import Path


class DataPoint:
    """Represents a single (date, value) pair for a time series."""

    def __init__(self, date_str: str, value_str: str) -> None:
        self.date = datetime.strptime(date_str, "%Y-%m-%d").date()
        self.value = Decimal(value_str)

    def __repr__(self) -> str:
        return f"DataPoint(date={self.date.isoformat()}, value={self.value})"


class Forex:
    """Loads and stores forex data from a CSV file, with query helpers."""

    def __init__(self, file_name: str) -> None:
        self.file_name = file_name
        self.data = self._process_data()

    def _process_data(self) -> list[DataPoint]:
        path = Path(self.file_name)
        if not path.exists():
            raise FileNotFoundError(f"File not found: {self.file_name!r}")

        points: list[DataPoint] = []
        with path.open() as f:
            reader = csv.reader(f)
            next(reader)  # skip header
            for date_str, value_str in reader:
                if value_str == ".":
                    continue
                points.append(DataPoint(date_str, value_str))
        return points

    # --------------------------
    # Query & aggregation methods
    # --------------------------
    def min_value(self) -> Decimal:
        """Return the minimum value in the series.

        Raises ValueError if there is no data.
        """
        if not self.data:
            raise ValueError("No data loaded")
        return min(self.data, key=lambda dp: dp.value).value

    def max_value(self) -> Decimal:
        """Return the maximum value in the series.

        Raises ValueError if there is no data.
        """
        if not self.data:
            raise ValueError("No data loaded")
        return max(self.data, key=lambda dp: dp.value).value

    def value_on(self, query_date: date) -> Decimal | None:
        """Return the value on `query_date`, or None if not found."""
        for dp in self.data:
            if dp.date == query_date:
                return dp.value
        return None

    def between(self, start_date: date, end_date: date) -> list[DataPoint]:
        """Return data points with dates between start_date and end_date (inclusive)."""
        if start_date > end_date:
            raise ValueError("start_date must be <= end_date")
        return [
            dp
            for dp in self.data
            if start_date <= dp.date <= end_date
        ]

    def moving_average(self, window: int) -> list[tuple[date, Decimal]]:
        """Return (date, average) pairs for a rolling window of given size.

        The first result is aligned with self.data[window - 1].
        """
        if window <= 0:
            raise ValueError("window must be positive")
        if window > len(self.data):
            raise ValueError("window cannot be larger than data length")

        result: list[tuple[date, Decimal]] = []
        # Simple implementation – recompute sum each time (could be optimized with a running sum)
        for end_idx in range(window - 1, len(self.data)):
            window_slice = self.data[end_idx - window + 1 : end_idx + 1]
            total = sum(dp.value for dp in window_slice)
            avg = total / Decimal(window)
            result.append((self.data[end_idx].date, avg))
        return result

    def __repr__(self) -> str:
        return f"Forex(file_name={self.file_name!r}, points={len(self.data)})"


# --------------------------
# Optional tests – only run if file exists
# --------------------------

sample_file = "DEXUSEU.csv"
if Path(sample_file).exists():
    fx = Forex(sample_file)
    assert fx.data, "Expected some data points in the CSV file"

    # Min / max sanity (just check they don't raise and have correct type)
    mn = fx.min_value()
    mx = fx.max_value()
    assert isinstance(mn, Decimal)
    assert isinstance(mx, Decimal)
    assert mn <= mx

    # value_on and between
    some_date = fx.data[0].date
    val = fx.value_on(some_date)
    assert val == fx.data[0].value

    sub = fx.between(some_date, some_date)
    assert len(sub) >= 1
    assert sub[0].date == some_date

    # moving_average
    if len(fx.data) >= 5:
        ma = fx.moving_average(3)
        assert len(ma) == len(fx.data) - 3 + 1
        # first moving average window
        first_window_vals = [dp.value for dp in fx.data[:3]]
        expected_avg = sum(first_window_vals) / Decimal(3)
        assert ma[0][1] == expected_avg

    fx  # show summary if run in a notebook
else:
    print("Forex tests skipped – file 'DEXUSEU.csv' not found in the working directory.")

## Problem 4 – Robust `SafeForex` Loader with Validation

In real data, we often have bad rows:

- Invalid dates
- Non-numeric values
- Empty lines
- Wrong number of columns

In this problem, you will implement a `SafeForex` class that:

1. Tries to parse each row into a `DataPoint`.
2. Skips rows that are malformed (but keeps track of them).
3. Exposes instance methods to inspect issues.

### Requirements

Design a `SafeForex` class that:

- Has attributes:
  - `data` – list of successfully parsed `DataPoint` objects.
  - `errors` – a list of **error records**, where each record is a small `dict` with keys:
    - `'line_number'`: the 1-based line number in the file (including header).
    - `'raw_row'`: the raw row as a tuple of strings.
    - `'error'`: a string representation of the exception that occurred.

- Methods:
  1. `load(self)` – instance method that reads the file and populates `data` and `errors`.
  2. `has_errors(self)` → `True` if any lines failed to parse.
  3. `error_summary(self)` → returns a dictionary with keys like `'total'`, `'first_n'` (first few errors), etc. (Design this as you see fit.)

- Behavior:
  - The constructor only stores `file_name` and initializes empty `data` and `errors`.
  - The actual I/O happens in `load` (an instance method), not in `__init__`.
  - The method should be robust: **no row failure should stop the entire load**.

### Hints

- Use `enumerate(reader, start=2)` to get line numbers (since line 1 is the header).
- Use `try` / `except` around parsing each row.
- Reuse the existing `DataPoint` class.

Implement this in the next cell. You can simulate bad rows by creating a small temporary CSV file (or by manually injecting bad lines if you wish).

In [8]:
# TODO: Implement SafeForex here.
# You can follow this skeleton or design your own.

class SafeForex:
    def __init__(self, file_name: str) -> None:
        self.file_name = file_name
        self.data = []      # list of DataPoint
        self.errors = []    # list of dicts as described

    def load(self):
        """Load data from the file, populating self.data and self.errors."""
        pass

    def has_errors(self) -> bool:
        pass

    def error_summary(self):
        pass

# Optional: create a small test CSV file in this notebook and test SafeForex.
# (You can use the `tempfile` module or write to a local file.)


### Solution 4 – Robust `SafeForex` Loader with Validation

Below is one possible implementation.

Key points:

- **Encapsulation**: `SafeForex` encapsulates both valid data and error information.
- The I/O and parsing logic live in `load`, keeping `__init__` simple.
- The `error_summary` method provides a compact diagnostic view of bad rows.

This style is common in production code: **never let one bad row crash the entire load**, but do record enough details to debug the issues later.

In [9]:
import csv
from pathlib import Path
from decimal import Decimal
from datetime import datetime
import tempfile


class DataPoint:
    """Simple reusable DataPoint definition (date + Decimal value)."""

    def __init__(self, date_str: str, value_str: str) -> None:
        self.date = datetime.strptime(date_str, "%Y-%m-%d").date()
        self.value = Decimal(value_str)

    def __repr__(self) -> str:
        return f"DataPoint(date={self.date.isoformat()}, value={self.value})"


class SafeForex:
    """Robust loader for forex data that records bad rows instead of failing."""

    def __init__(self, file_name: str) -> None:
        self.file_name = file_name
        self.data: list[DataPoint] = []
        self.errors: list[dict] = []

    def load(self) -> None:
        """Load data from the file, populating self.data and self.errors.

        This method can be called multiple times; it will overwrite previous
        contents of data and errors.
        """
        self.data.clear()
        self.errors.clear()

        path = Path(self.file_name)
        if not path.exists():
            raise FileNotFoundError(f"File not found: {self.file_name!r}")

        with path.open() as f:
            reader = csv.reader(f)
            header = next(reader, None)  # header, may be None for empty file

            for line_number, row in enumerate(reader, start=2):
                # Expecting 2 columns: date, value
                try:
                    if len(row) != 2:
                        raise ValueError(f"Expected 2 columns, got {len(row)}")

                    date_str, value_str = row

                    if value_str == ".":
                        # Missing value – choose to treat as error or skip
                        raise ValueError("Missing value '.' encountered")

                    dp = DataPoint(date_str, value_str)
                    self.data.append(dp)
                except Exception as exc:  # noqa: BLE001
                    self.errors.append(
                        {
                            "line_number": line_number,
                            "raw_row": tuple(row),
                            "error": str(exc),
                        }
                    )

    def has_errors(self) -> bool:
        """Return True if any rows failed to parse."""
        return bool(self.errors)

    def error_summary(self, max_items: int = 3) -> dict:
        """Return a small summary of parsing errors.

        Parameters
        ----------
        max_items : int
            Maximum number of error records to include in the 'sample' list.
        """
        return {
            "total": len(self.errors),
            "sample": self.errors[:max_items],
        }

    def __repr__(self) -> str:
        return (
            f"SafeForex(file_name={self.file_name!r}, "
            f"points={len(self.data)}, errors={len(self.errors)})"
        )


# --------------------------
# Small self-contained test using a temporary CSV file
# --------------------------

csv_content = """DATE,VALUE
2015-01-01,1.0
2015-01-02,1.1
bad-date,1.2
2015-01-04,not-a-number
2015-01-05,.
2015-01-06,1.3
too,many,columns
"""

with tempfile.NamedTemporaryFile("w+", suffix=".csv", delete=False) as tf:
    tf.write(csv_content)
    temp_name = tf.name

safe_fx = SafeForex(temp_name)
safe_fx.load()

# We expect some good rows and some bad rows
assert len(safe_fx.data) > 0
assert safe_fx.has_errors() is True
summary = safe_fx.error_summary()
assert summary["total"] == len(safe_fx.errors)

safe_fx, summary

(SafeForex(file_name='C:\\Users\\user1\\AppData\\Local\\Temp\\tmp26thpry8.csv', points=3, errors=4),
 {'total': 4,
  'sample': [{'line_number': 4,
    'raw_row': ('bad-date', '1.2'),
    'error': "time data 'bad-date' does not match format '%Y-%m-%d'"},
   {'line_number': 5,
    'raw_row': ('2015-01-04', 'not-a-number'),
    'error': "[<class 'decimal.ConversionSyntax'>]"},
   {'line_number': 6,
    'raw_row': ('2015-01-05', '.'),
    'error': "Missing value '.' encountered"}]})